# Model Performance Transformations

Lets practice some basic data transformation for ML performance enhancement

In [1]:
import tensorflow as tf

print("Versión de tensorflow:{}".format(tf.__version__))
print("GPU:{}".format(tf.test.gpu_device_name()))

Versión de tensorflow:2.1.0
GPU:/device:GPU:0


In [2]:
import tensorflow as tf
from tensorflow.python.client import device_lib

print(tf.__version__)

num_gpus = len(tf.config.list_physical_devices('GPU'))
print(num_gpus)
print(device_lib.list_local_devices())


2.1.0
1
[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 8179629287542703050
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 3035942094
locality {
  bus_id: 1
  links {
  }
}
incarnation: 2314646193040995789
physical_device_desc: "device: 0, name: NVIDIA GeForce GTX 1650, pci bus id: 0000:01:00.0, compute capability: 7.5"
]


In [3]:
# Imports
#%pip install duckdb
#%pip install sqlite
import tensorflow as tf
import keras
import pandas as pd
import numpy as np 
import sys
import re
sys.path.insert(0, "modules")
import seaborn as sns
import pickle
import os
from sklearn.preprocessing import RobustScaler, StandardScaler
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import SGDClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn import linear_model
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import Ridge
from sklearn.svm import SVR
from sklearn.linear_model import SGDRegressor
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import GradientBoostingRegressor



Using TensorFlow backend.


In [248]:
diamonds = pd.read_csv('../data/diamonds_train_data/diamonds.csv')

In [249]:
from functions import cat_var,delete_features_corr, remove_outliers, remove_outliers_price, drop_zeros, remove_duplicates, imputation, label_encoder, onehot_encoder,  inverse_scaler, feature_ing,  robust_scaling_train, stardard_scaling_train, transformation_data, calculate_log, remove_outliers

In [250]:
diamonds = diamonds.drop(columns=['city','depth'])

In [251]:
diamonds.describe()

,price,carat,table,x,y,z
count,40429.000000,40429.000000,40429.000000,40429.000000,40429.000000,40429.000000
mean,3926.306340,0.797380,57.445579,5.729556,5.732845,3.538522
std,3989.522774,0.475125,2.233075,1.122265,1.144897,0.693548
min,326.000000,0.200000,43.000000,3.770000,3.720000,1.070000
25%,945.000000,0.400000,56.000000,4.710000,4.720000,2.910000
50%,2396.000000,0.700000,57.000000,5.690000,5.710000,3.520000
75%,5329.000000,1.040000,59.000000,6.540000,6.540000,4.040000
max,18823.000000,4.500000,95.000000,10.230000,58.900000,8.060000


In [252]:
#diamonds.sort_values(by='y').tail(50)

## Transformations


In [253]:
#diamonds= diamonds[diamonds['price'] <= 15000]
diamonds = diamonds[diamonds['carat'] <= 2.5]
diamonds = diamonds[diamonds['y'] <= 8.5]
diamonds = diamonds[diamonds['x'] <= 8.5]
diamonds = diamonds[diamonds['z'] <= 5.5]
#diamonds = diamonds[diamonds['table'] <= 60]
#diamonds = diamonds[diamonds['depth'] <= 61]
diamonds

,cut,color,clarity,price,carat,table,x,y,z
0,Very Good,H,VS2,505,0.32,57.0,4.35,4.38,2.75
1,Very Good,G,VS1,605,0.30,56.0,4.29,4.33,2.66
2,Very Good,H,SI1,7066,1.55,57.0,7.35,7.31,4.64
3,Very Good,H,VS1,2422,0.70,56.0,5.65,5.68,3.52
4,Very Good,H,SI1,2553,0.80,54.0,5.93,5.88,3.73
...,...,...,...,...,...,...,...,...,...
40424,Fair,F,SI2,3633,1.02,58.0,6.21,6.14,4.11
40425,Fair,H,SI1,4758,1.02,60.0,6.29,6.22,4.10
40426,Fair,F,SI2,4036,1.00,67.0,6.48,6.41,3.80
40427,Fair,F,SI1,11154,1.50,60.0,7.18,7.10,4.61


In [254]:
dic = {
    'Ideal': 1,
    'Very Good': 1.161889072,
    'Good': 1.120914073,
    'Premium': 1.328489204,
    'Fair': 1.222096908,
    'E': 1,
    'D': 1.014189154,
    'G': 1.29667896,
    'F': 1.188876741,
    'H': 1.420910907,
    'I': 1.590453442,
    'J': 1.612942016,
    'IF': 1.093142836,
    'VVS1': 1,
    'VVS2': 1.266255428,
    'VS1': 1.473882215,
    'VS2': 1.515365621,
    'SI1': 1.542725208,
    'I1': 1.364055654,
    'SI2': 1.91083619
}

# Definir la función de etiquetado de datos
def data_labeling(df, cols, dic):
    df = df.copy()
    for col in cols:
        if col in df.columns:
            df[col] = df[col].map(dic).astype(float)
    return df

# Aplicar la función de etiquetado de datos
diamonds = data_labeling(diamonds, ['cut', 'color', 'clarity'], dic)
diamonds

,cut,color,clarity,price,carat,table,x,y,z
0,1.161889,1.420911,1.515366,505,0.32,57.0,4.35,4.38,2.75
1,1.161889,1.296679,1.473882,605,0.30,56.0,4.29,4.33,2.66
2,1.161889,1.420911,1.542725,7066,1.55,57.0,7.35,7.31,4.64
3,1.161889,1.420911,1.473882,2422,0.70,56.0,5.65,5.68,3.52
4,1.161889,1.420911,1.542725,2553,0.80,54.0,5.93,5.88,3.73
...,...,...,...,...,...,...,...,...,...
40424,1.222097,1.188877,1.910836,3633,1.02,58.0,6.21,6.14,4.11
40425,1.222097,1.420911,1.542725,4758,1.02,60.0,6.29,6.22,4.10
40426,1.222097,1.188877,1.910836,4036,1.00,67.0,6.48,6.41,3.80
40427,1.222097,1.188877,1.542725,11154,1.50,60.0,7.18,7.10,4.61


In [244]:
#diamonds=remove_outliers(diamonds)
#diamonds = diamonds.query('z != 0')
diamonds=imputation(diamonds)

def feature_xy(x, y):
    try:
        return x / y


def feature_ad(carat, x, y, z):
    try:
        return carat / (x * y * z)
    
diamonds['x/y'] = (diamonds.apply(lambda x: feature_xy(x['x'], x['y']), axis=1))**12
#diamonds['td'] = diamonds.apply(lambda x: feature_xy(x['table'], x['depth']), axis=1)
diamonds['volum'] =  diamond['x']* diamond['y']* diamond['z'])
diamonds['density'] = diamond['carat']/diamonds['volum']
diamonds['cc'] = diamonds['carat'] * diamonds['clarity']
#diamonds['colcut'] = diamonds['color'] * diamonds['cut']
#diamonds['colcla'] = diamonds['color'] * diamonds['clarity']
diamonds['ccc'] = diamonds['color'] * diamonds['clarity'] * diamonds['cut']
diamonds = diamonds.drop(columns=['y'])


IndentationError: unexpected unindent (<ipython-input-244-366585041f21>, line 10)

In [261]:
diamonds = diamonds.drop(columns=['y','table'])

In [262]:
diamonds

,cut,color,clarity,price,carat,x,z
0,1.161889,1.420911,1.515366,505,0.32,4.35,2.75
1,1.161889,1.296679,1.473882,605,0.30,4.29,2.66
2,1.161889,1.420911,1.542725,7066,1.55,7.35,4.64
3,1.161889,1.420911,1.473882,2422,0.70,5.65,3.52
4,1.161889,1.420911,1.542725,2553,0.80,5.93,3.73
...,...,...,...,...,...,...,...
40424,1.222097,1.188877,1.910836,3633,1.02,6.21,4.11
40425,1.222097,1.420911,1.542725,4758,1.02,6.29,4.10
40426,1.222097,1.188877,1.910836,4036,1.00,6.48,3.80
40427,1.222097,1.188877,1.542725,11154,1.50,7.18,4.61


In [263]:
X = diamonds.drop(columns=['price'])
y = diamonds['price']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)



In [264]:
model = GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, max_depth=3, random_state=42)
model.fit(X_train, y_train)

# Evaluar el modelo
predictions = model.predict(X_test)
mse = mean_squared_error(y_test, predictions)
rmse = np.sqrt(mse)
rmse

664.3056778471555

In [121]:
#filename = '../models/GradientBoostin004.sav'
pickle.dump(model, open(filename, 'wb'))
print('Your model has been saved with  pickle!!!')

Your model has been saved with  pickle!!!


In [122]:
# Definir el modelo
model = GradientBoostingRegressor(random_state=42)

# Definir los hiperparámetros a buscar
param_grid = {
    'n_estimators': [32, 64, 128],
    'learning_rate': [0.01, 0.05, 0.1, 0.2],
    'max_depth': [3, 4, 5],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'subsample': [0.8, 0.9, 1.0]
}

# Realizar la búsqueda
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=3, scoring='neg_mean_squared_error', n_jobs=-1)
grid_search.fit(X_train, y_train)

# Mejor modelo y sus hiperparámetros
best_model = grid_search.best_estimator_
print(f'Mejores hiperparámetros: {grid_search.best_params_}')

# Evaluar el mejor modelo
predictions = best_model.predict(X_test)
mse = mean_squared_error(y_test, predictions)
rmse = np.sqrt(mse)
print(f'Root Mean Squared Error (RMSE) del mejor modelo: {rmse:.4f}')

# Evaluar el modelo con validación cruzada
scores = cross_val_score(best_model, X_train, y_train, cv=5, scoring='neg_mean_squared_error')
rmse_scores = np.sqrt(-scores)
print(f'RMSE scores: {rmse_scores}')
print(f'Mean RMSE: {rmse_scores.mean():.4f}')
print(f'Standard Deviation of RMSE: {rmse_scores.std():.4f}')


KeyboardInterrupt: 

In [ ]:
#filename = '../models/GradientBoostin003.sav'
pickle.dump(grid_search, open(filename, 'wb'))
print('Your model has been saved with  pickle!!!')

In [23]:
import xgboost as xgb
from sklearn.model_selection import GridSearchCV

model = xgb.XGBRegressor(tree_method='gpu_hist', predictor='gpu_predictor', random_state=42)
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 3, 10],
    'min_samples_split': [2, 10],
    'min_samples_leaf': [1, 4],
    'max_features': [None, 'sqrt', 'log2']
}

grid_search = GridSearchCV(model, param_grid, cv=5, verbose=3, scoring='neg_root_mean_squared_error', n_jobs=-1)
grid_search.fit(X, y)


print('\n')
print('Best hyperparameters: ', grid_search.best_params_, '\n')
print('Best score: ', -grid_search.best_score_, '\n')

Fitting 5 folds for each of 108 candidates, totalling 540 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done  96 tasks      | elapsed:  3.4min
[Parallel(n_jobs=-1)]: Done 256 tasks      | elapsed:  6.4min
[Parallel(n_jobs=-1)]: Done 480 tasks      | elapsed: 41.7min
[Parallel(n_jobs=-1)]: Done 540 out of 540 | elapsed: 58.7min finished


[13:31:16] WARNING: C:/Users/administrator/workspace/xgboost-win64_release_1.6.0/src/learner.cc:627: 
Parameters: { "min_samples_leaf", "min_samples_split" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.




Best hyperparameters:  {'max_depth': 3, 'max_features': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 300} 

Best score:  423.5543582794932 



In [88]:
filename = '../models/XGBoostin005.sav'
pickle.dump(grid_search, open(filename, 'wb'))
print('Your model has been saved with  pickle!!!')

Your model has been saved with  pickle!!!


---

In [103]:
diamonds_test= pd.read_csv('../data/diamonds_test_data/diamonds_test.csv')

In [104]:
diamonds_test

,id,carat,cut,color,clarity,depth,table,x,y,z,city
0,0,0.79,Very Good,F,SI1,62.7,60.0,5.82,5.89,3.67,Amsterdam
1,1,1.20,Ideal,J,VS1,61.0,57.0,6.81,6.89,4.18,Surat
2,2,1.57,Premium,H,SI1,62.2,61.0,7.38,7.32,4.57,Kimberly
3,3,0.90,Very Good,F,SI1,63.8,54.0,6.09,6.13,3.90,Kimberly
4,4,0.50,Very Good,F,VS1,62.9,58.0,5.05,5.09,3.19,Amsterdam
...,...,...,...,...,...,...,...,...,...,...,...
13480,13480,0.57,Ideal,E,SI1,61.9,56.0,5.35,5.32,3.30,Amsterdam
13481,13481,0.71,Ideal,I,VS2,62.2,55.0,5.71,5.73,3.56,New York City
13482,13482,0.70,Ideal,F,VS1,61.6,55.0,5.75,5.71,3.53,Tel Aviv
13483,13483,0.70,Very Good,F,SI2,58.8,57.0,5.85,5.89,3.45,Surat


In [105]:

dic = {'Ideal': 1, 'Very Good': 1.17, 'Good': 1.21, 'Premium': 1.26, 'Fair': 1.32,
       'E': 1, 'D': 1.03, 'G': 1.22, 'F': 1.15, 'H': 1.32, 'I': 1.36, 'J': 1,
       'IF': 1, 'VVS1': 1.03, 'VVS2': 1.33, 'VS1': 1.60, 'VS2': 1.63, 'SI1': 1.70,
       'I1': 1.76, 'SI2': 2.04}

# Definir la función de etiquetado de datos
def data_labeling(df, cols, dic):
    df = df.copy()
    for col in cols:
        if col in df.columns:
            df[col] = df[col].map(dic).astype(float)
    return df

# Aplicar la función de etiquetado de datos
diamonds_test = data_labeling(diamonds_test, ['cut', 'color', 'clarity'], dic)
diamonds_test

,id,carat,cut,color,clarity,depth,table,x,y,z,city
0,0,0.79,1.17,1.15,1.70,62.7,60.0,5.82,5.89,3.67,Amsterdam
1,1,1.20,1.00,1.00,1.60,61.0,57.0,6.81,6.89,4.18,Surat
2,2,1.57,1.26,1.32,1.70,62.2,61.0,7.38,7.32,4.57,Kimberly
3,3,0.90,1.17,1.15,1.70,63.8,54.0,6.09,6.13,3.90,Kimberly
4,4,0.50,1.17,1.15,1.60,62.9,58.0,5.05,5.09,3.19,Amsterdam
...,...,...,...,...,...,...,...,...,...,...,...
13480,13480,0.57,1.00,1.00,1.70,61.9,56.0,5.35,5.32,3.30,Amsterdam
13481,13481,0.71,1.00,1.36,1.63,62.2,55.0,5.71,5.73,3.56,New York City
13482,13482,0.70,1.00,1.15,1.60,61.6,55.0,5.75,5.71,3.53,Tel Aviv
13483,13483,0.70,1.17,1.15,2.04,58.8,57.0,5.85,5.89,3.45,Surat


In [106]:
def feature_xy(x, y):
    try:
        return x / y
    except:
        return 0.999

def feature_ad(carat, x, y, z):
    try:
        return carat / (x * y * z)
    except:
        return 0.
    

diamonds_test['x/y'] = (diamonds_test.apply(lambda x: feature_xy(x['x'], x['y']), axis=1))**12
diamonds_test['td'] = diamonds_test.apply(lambda x: feature_xy(x['table'], x['depth']), axis=1)
diamonds_test['ad'] = diamonds_test.apply(lambda x: feature_ad(x['carat'], x['x'], x['y'], x['z']), axis=1)
diamonds_test['cc'] = diamonds_test['carat'] * diamonds_test['clarity']
diamonds_test['colcut'] =diamonds_test['color'] * diamonds_test['cut']
diamonds_test['colcla'] = diamonds_test['color'] * diamonds_test['clarity']
diamonds_test['ccc'] = diamonds_test['color'] * diamonds_test['clarity'] * diamonds_test['cut']


diamonds_test = diamonds_test.drop(columns=['id'])
diamonds_test

,carat,cut,color,clarity,depth,table,x,y,z,city,x/y,td,ad,cc,colcut,colcla,ccc
0,0.79,1.17,1.15,1.70,62.7,60.0,5.82,5.89,3.67,Amsterdam,0.866348,0.956938,0.006279,1.3430,1.3455,1.9550,2.28735
1,1.20,1.00,1.00,1.60,61.0,57.0,6.81,6.89,4.18,Surat,0.869230,0.934426,0.006118,1.9200,1.0000,1.6000,1.60000
2,1.57,1.26,1.32,1.70,62.2,61.0,7.38,7.32,4.57,Kimberly,1.102918,0.980707,0.006359,2.6690,1.6632,2.2440,2.82744
3,0.90,1.17,1.15,1.70,63.8,54.0,6.09,6.13,3.90,Kimberly,0.924447,0.846395,0.006182,1.5300,1.3455,1.9550,2.28735
4,0.50,1.17,1.15,1.60,62.9,58.0,5.05,5.09,3.19,Amsterdam,0.909668,0.922099,0.006098,0.8000,1.3455,1.8400,2.15280
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13480,0.57,1.00,1.00,1.70,61.9,56.0,5.35,5.32,3.30,Amsterdam,1.069808,0.904685,0.006069,0.9690,1.0000,1.7000,1.70000
13481,0.71,1.00,1.36,1.63,62.2,55.0,5.71,5.73,3.56,New York City,0.958910,0.884244,0.006096,1.1573,1.3600,2.2168,2.21680
13482,0.70,1.00,1.15,1.60,61.6,55.0,5.75,5.71,3.53,Tel Aviv,1.087379,0.892857,0.006040,1.1200,1.1500,1.8400,1.84000
13483,0.70,1.17,1.15,2.04,58.8,57.0,5.85,5.89,3.45,Surat,0.921482,0.969388,0.005889,1.4280,1.3455,2.3460,2.74482


In [107]:
neworder = diamonds.columns.tolist()
neworder.remove('price')

diamonds_test = diamonds_test.reindex(columns=neworder)
diamonds_test 

,cut,color,clarity,carat,depth,table,x,y,z,x/y,td,ad,cc,colcut,colcla,ccc
0,1.17,1.15,1.70,0.79,62.7,60.0,5.82,5.89,3.67,0.866348,0.956938,0.006279,1.3430,1.3455,1.9550,2.28735
1,1.00,1.00,1.60,1.20,61.0,57.0,6.81,6.89,4.18,0.869230,0.934426,0.006118,1.9200,1.0000,1.6000,1.60000
2,1.26,1.32,1.70,1.57,62.2,61.0,7.38,7.32,4.57,1.102918,0.980707,0.006359,2.6690,1.6632,2.2440,2.82744
3,1.17,1.15,1.70,0.90,63.8,54.0,6.09,6.13,3.90,0.924447,0.846395,0.006182,1.5300,1.3455,1.9550,2.28735
4,1.17,1.15,1.60,0.50,62.9,58.0,5.05,5.09,3.19,0.909668,0.922099,0.006098,0.8000,1.3455,1.8400,2.15280
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13480,1.00,1.00,1.70,0.57,61.9,56.0,5.35,5.32,3.30,1.069808,0.904685,0.006069,0.9690,1.0000,1.7000,1.70000
13481,1.00,1.36,1.63,0.71,62.2,55.0,5.71,5.73,3.56,0.958910,0.884244,0.006096,1.1573,1.3600,2.2168,2.21680
13482,1.00,1.15,1.60,0.70,61.6,55.0,5.75,5.71,3.53,1.087379,0.892857,0.006040,1.1200,1.1500,1.8400,1.84000
13483,1.17,1.15,2.04,0.70,58.8,57.0,5.85,5.89,3.45,0.921482,0.969388,0.005889,1.4280,1.3455,2.3460,2.74482


In [108]:
filename = '../models/GradientBoostin004.sav'
model = pickle.load(open(filename, 'rb'))


In [109]:

predictions=model.predict(diamonds_test)
df_predictions = pd.DataFrame(predictions, columns=['price'], index=range(len(predictions)))
df_predictions = df_predictions.rename_axis('id')
df_predictions

,price
id,
0,2950.828003
1,7082.180899
2,8827.457354
3,3933.062101
4,1651.184185
...,...
13480,1609.053051
13481,2304.225876
13482,3051.377224


In [110]:
df_predictions.to_csv('../data/submission/004.csv')

## Enconding

- [OneHotEncoder](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html#sklearn.preprocessing.OneHotEncoder)

- [LabelEncoder](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.LabelEncoder.html#sklearn.preprocessing.LabelEncoder)


In [80]:
#diamonds=onehot_encoder(diamonds)


In [137]:
diamonds =label_encoder(diamonds)

In [138]:
diamonds

,cut,color,clarity,price,carat,depth,table,x,y,z
0,4,4,5,505,0.32,63.0,57.0,4.35,4.38,2.75
1,4,3,4,605,0.30,61.7,56.0,4.29,4.33,2.66
2,4,4,2,7066,1.55,63.2,57.0,7.35,7.31,4.64
3,4,4,4,2422,0.70,62.1,56.0,5.65,5.68,3.52
4,4,4,2,2553,0.80,63.2,54.0,5.93,5.88,3.73
...,...,...,...,...,...,...,...,...,...,...
40424,0,2,3,3633,1.02,66.6,58.0,6.21,6.14,4.11
40425,0,4,2,4758,1.02,65.5,60.0,6.29,6.22,4.10
40426,0,2,3,4036,1.00,59.0,67.0,6.48,6.41,3.80
40427,0,2,2,11154,1.50,64.6,60.0,7.18,7.10,4.61


### Feture Encoding

In [139]:


#diamonds = calculate_log(diamonds, 'carat')
#diamonds['x']=diamonds['x']**2
#diamonds = calculate_log(diamonds, 'x')
#diamonds = calculate_log(diamonds, 'y')
#diamonds = calculate_log(diamonds, 'z')
#diamonds['ratio_length_width'] = trans_df['x']/trans_df['y']
diamonds['ratio_length_width_depth'] = diamonds['x']/diamonds['y']/diamonds['z']
diamonds['volume'] = diamonds['x']*diamonds['y']*diamonds['z']
diamonds['density'] = diamonds['carat']/diamonds['volume']**10



#diamonds['x/y'] = (diamonds['x']/ diamonds['y'])
diamonds['td'] = (diamonds['table']/ diamonds['depth'])

diamonds['cc'] = diamonds['carat'] * diamonds['clarity']
#diamonds['colcut'] = diamonds['color'] * diamonds['cut']
#diamonds['colcla'] = diamonds['color'] * diamonds['clarity']
diamonds['ccc'] = diamonds['color'] * diamonds['clarity'] * diamonds['cut']
diamonds = diamonds.drop(columns=['depth','table'])


In [140]:
def remove_outliers(df, deviations=3):

    df_clean = df.copy()
    for col in df.columns:
        if pd.api.types.is_numeric_dtype(df[col]):
            col_mean = df[col].mean()
            col_std = df[col].std()
            min_val = col_mean - deviations * col_std
            max_val = col_mean + deviations * col_std
            df_clean = df_clean[(df_clean[col] >= min_val) & (df_clean[col] <= max_val)]
    return df_clean
remove_outliers(diamonds, deviations=3)

,cut,color,clarity,price,carat,x,y,z,ratio_length_width_depth,volume,density,td,cc,ccc
0,4,4,5,505,0.32,4.35,4.38,2.75,0.361146,52.395750,2.052057e-18,0.904762,1.60,80
1,4,3,4,605,0.30,4.29,4.33,2.66,0.372467,49.411362,3.458223e-18,0.907618,1.20,48
2,4,4,2,7066,1.55,7.35,7.31,4.64,0.216697,249.300240,1.671494e-24,0.901899,3.10,32
3,4,4,4,2422,0.70,5.65,5.68,3.52,0.282590,112.963840,2.068729e-21,0.901771,2.80,64
4,4,4,2,2553,0.80,5.93,5.88,3.73,0.270376,130.059132,5.776722e-22,0.854430,1.60,32
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40423,0,3,2,3631,1.00,6.00,6.15,4.11,0.237375,151.659000,1.553525e-22,0.841950,2.00,0
40424,0,2,3,3633,1.02,6.21,6.14,4.11,0.246083,156.711834,1.141779e-22,0.870871,3.06,0
40425,0,4,2,4758,1.02,6.29,6.22,4.10,0.246647,160.407580,9.043807e-23,0.916031,2.04,0
40427,0,2,2,11154,1.50,7.18,7.10,4.61,0.219364,235.008580,2.919134e-24,0.928793,3.00,0


## Scaling



In [46]:
#diamonds=stardard_scaling_train(diamonds)


In [141]:
diamonds=robust_scaling_train(diamonds)

In [142]:
diamonds

,cut,color,clarity,carat,x,y,z,ratio_length_width_depth,volume,density,td,cc,ccc,price
0,2.0,0.333333,0.333333,-0.593750,-0.732240,-0.730769,-0.681416,0.807000,-0.588471,6.866661,-0.334367,-0.319249,2.142857,505
1,2.0,0.000000,0.000000,-0.625000,-0.765027,-0.758242,-0.761062,0.924896,-0.616652,11.576118,-0.283953,-0.507042,1.000000,605
2,2.0,0.333333,-0.666667,1.328125,0.907104,0.879121,0.991150,-0.697245,1.270813,-0.005973,-0.384915,0.384977,0.428571,7066
3,2.0,0.333333,0.000000,0.000000,-0.021858,-0.016484,0.000000,-0.011048,-0.016553,0.000950,-0.387164,0.244131,1.571429,2422
4,2.0,0.333333,-0.666667,0.156250,0.131148,0.093407,0.185841,-0.138243,0.144870,-0.004044,-1.222949,-0.319249,0.428571,2553
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40424,-2.0,-0.333333,-0.333333,0.500000,0.284153,0.236264,0.522124,-0.391225,0.396540,-0.005597,-0.932699,0.366197,-0.714286,3633
40425,-2.0,0.333333,-0.666667,0.500000,0.327869,0.280220,0.513274,-0.385348,0.431438,-0.005676,-0.135424,-0.112676,-0.714286,4758
40426,-2.0,-0.333333,-0.333333,0.468750,0.431694,0.384615,0.247788,-0.183485,0.407191,-0.005630,3.740862,0.338028,-0.714286,4036
40427,-2.0,-0.333333,-0.666667,1.250000,0.814208,0.763736,0.964602,-0.669468,1.135863,-0.005969,0.089884,0.338028,-0.714286,11154


## Train-Test-Split


In [143]:
X = diamonds.drop('price', axis = 1)
y = diamonds['price']
print(X.shape,y.shape)

(40429, 13) (40429,)


In [144]:
# Train + test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print(f"X_train: {X_train.shape}, X_test: {X_test.shape}, y_train: {y_train.shape}, y_test: {y_test.shape}")
print(f"X_train: {type(X_train)}, X_test: {type(X_test)}, y_train: {type(y_train)}, y_test: {type(y_test)}")


X_train: (32343, 13), X_test: (8086, 13), y_train: (32343,), y_test: (8086,)
X_train: <class 'pandas.core.frame.DataFrame'>, X_test: <class 'pandas.core.frame.DataFrame'>, y_train: <class 'pandas.core.series.Series'>, y_test: <class 'pandas.core.series.Series'>


## MODELS

### XGBOOST


In [ ]:
import xgboost as xgb
from sklearn.model_selection import GridSearchCV

model = xgb.XGBRegressor(tree_method='gpu_hist', predictor='gpu_predictor', random_state=42)
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 3, 10],
    'min_samples_split': [2, 10],
    'min_samples_leaf': [1, 4],
    'max_features': [None, 'sqrt', 'log2']
}

grid_search = GridSearchCV(model, param_grid, cv=5, verbose=3, scoring='neg_root_mean_squared_error', n_jobs=-1)
grid_search.fit(X, y)


print('\n')
print('Best hyperparameters: ', grid_search.best_params_, '\n')
print('Best score: ', -grid_search.best_score_, '\n')

Fitting 5 folds for each of 108 candidates, totalling 540 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done  96 tasks      | elapsed:  3.3min
[Parallel(n_jobs=-1)]: Done 256 tasks      | elapsed:  6.5min
[Parallel(n_jobs=-1)]: Done 480 tasks      | elapsed: 89.7min


In [131]:
#filename = '../models/xgboost_003.sav'
pickle.dump(grid_search, open(filename, 'wb'))
print('Your model has been saved with  pickle!!!')

Your model has been saved with  pickle!!!


### GradientBoosting

In [21]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error

# Definir el modelo
model = GradientBoostingRegressor(random_state=42)

# Definir el espacio de búsqueda de hiperparámetros
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [3, 5, 7],
    'learning_rate': [0.1, 0.05, 0.01],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['auto', 'sqrt', 'log2']
}

# Realizar la búsqueda de hiperparámetros
grid_search = GridSearchCV(model, param_grid, cv=5, verbose=3, scoring='neg_mean_squared_error', n_jobs=-1)
grid_search.fit(X, y)

# Imprimir los mejores hiperparámetros y puntuación
print('\n')
print('Mejores hiperparámetros: ', grid_search.best_params_, '\n')
print('Mejor puntuación (neg_mean_squared_error): ', grid_search.best_score_, '\n')

# Obtener la raíz cuadrada del error cuadrático medio (RMSE)
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X)
rmse = mean_squared_error(y, y_pred, squared=False)
print('RMSE en conjunto de entrenamiento: ', rmse)


Fitting 5 folds for each of 729 candidates, totalling 3645 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done  96 tasks      | elapsed:  2.4min
[Parallel(n_jobs=-1)]: Done 256 tasks      | elapsed:  4.5min
[Parallel(n_jobs=-1)]: Done 480 tasks      | elapsed:  9.2min
[Parallel(n_jobs=-1)]: Done 768 tasks      | elapsed: 15.5min
[Parallel(n_jobs=-1)]: Done 1120 tasks      | elapsed: 28.3min
[Parallel(n_jobs=-1)]: Done 1536 tasks      | elapsed: 35.8min
[Parallel(n_jobs=-1)]: Done 2016 tasks      | elapsed: 46.7min
[Parallel(n_jobs=-1)]: Done 2560 tasks      | elapsed: 63.8min
[Parallel(n_jobs=-1)]: Done 3168 tasks      | elapsed: 75.1min
[Parallel(n_jobs=-1)]: Done 3645 out of 3645 | elapsed: 89.1min finished




Mejores hiperparámetros:  {'learning_rate': 0.05, 'max_depth': 7, 'max_features': 'auto', 'min_samples_leaf': 2, 'min_samples_split': 5, 'n_estimators': 300} 

Mejor puntuación (neg_mean_squared_error):  -311521.62178227614 

RMSE en conjunto de entrenamiento:  380.71156748464233


In [130]:
#filename = '../models/Gradientboosting_002.sav'
pickle.dump(grid_search, open(filename, 'wb'))
print('Your model has been saved with  pickle!!!')

Your model has been saved with  pickle!!!


### RandomForestRegressor

In [54]:
model = RandomForestRegressor(random_state = 42)

param_grid = {'n_estimators': [100, 200, 300],  # Number of trees in the forest.
              'max_depth': [None, 3, 10],  # Maximum depth of the trees.
              'min_samples_split': [2, 10],  # Minimum number of samples required to split an internal node.
              'min_samples_leaf': [1, 4],  # Minimum number of samples required to be at a leaf node.
              'max_features': [None, 'sqrt', 'log2']  # Number of features to consider when looking for the best split.
              }
grid_search = GridSearchCV(model,
                           param_grid,
                           cv=5,
                           verbose=3,
                           scoring='neg_root_mean_squared_error',
                           n_jobs=-1)

grid_search.fit(X,y)

print('\n')
print('Best hyperparameters: ', grid_search.best_params_, '\n')
print('Best score: ', -grid_search.best_score_, '\n')

Fitting 5 folds for each of 108 candidates, totalling 540 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done  96 tasks      | elapsed:  6.4min
[Parallel(n_jobs=-1)]: Done 256 tasks      | elapsed:  9.5min
[Parallel(n_jobs=-1)]: Done 480 tasks      | elapsed: 14.6min
[Parallel(n_jobs=-1)]: Done 540 out of 540 | elapsed: 15.4min finished




Best hyperparameters:  {'max_depth': None, 'max_features': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 300} 

Best score:  666.5245666065281 



In [55]:
#filename = '../models/RandomForest_001.sav'
pickle.dump(grid_search, open(filename, 'wb'))
print('Your model has been saved with  pickle!!!')

Your model has been saved with  pickle!!!


### GradientBoostingRegressor


In [ ]:
model = GradientBoostingRegressor(random_state = 42)


param_grid_GB = {'n_estimators': [100, 500],  # Number of boosting stages to be run.
             'learning_rate': [0.01, 0.1],  # Rate at which the contribution of each tree is shrunk.
              'max_depth': [None, 3, 10],  # Maximum depth of the individual regression estimators.
              'min_samples_split': [2, 10],  # Minimum number of samples required to split an internal node.
              'min_samples_leaf': [1, 4],  # Minimum number of samples required to be at a leaf node.
              'max_features': [None, 'sqrt', 'log2']  # The number of features to consider when looking for the best split.
              }

grid_search = GridSearchCV(model,
                           param_grid_GB,
                           cv=5,
                           verbose=3,
                           scoring='neg_root_mean_squared_error',
                           n_jobs=-1)


grid_search.fit(X,y)

print('\n')
print('Best hyperparameters: ', grid_search.best_params_, '\n')
print('Best score: ', -grid_search.best_score_, '\n')

In [ ]:
#filename = '../models/Grandient_003.sav'
pickle.dump(grid_search, open(filename, 'wb'))
print('Your model has been saved with  pickle!!!')


### Linearmodel

In [53]:
#Linear model
my_linear_model = LinearRegression()
linear_param = my_linear_model.fit(X_train, y_train)
linear_pred = my_linear_model.predict(X_test)
rmse = mean_squared_error(y_test, linear_pred, squared=False)
print(f"Linear model RMSE is: {rmse}")

Linear model RMSE is: 1108.5648335329738


In [ ]:

#filename = '../models/linearregression_001.sav'
pickle.dump(my_linear_model, open(filename, 'wb'))
print('Your model has been saved with  pickle!!!')